# XGBoost = Extreme Gradient Boosting 모델


## XGBoost에서 개선된 점
- 자체적으로 결측치를 처리한다
- 속도를 향상한다. 
  - 근사 분할 탐색 알고리즘: 평범한 결정 트리는 그리디 알고리즘을 사용하지만, xgboost에서는 데이터를 나누는 퍼센트인 분위수 (퀀타일)을 사용해서 후보 분할을 제안한다. 
  - 희소성 고려 분할 탐색
  - 병렬 컴퓨팅: 부스팅은 이전 트리의 결과에 의존하기 때문에 병렬 컴퓨팅이 어렵다. 병렬 컴퓨팅이란 여러 개의 계산 유닛이 동시에 동일한 작업을 하는 것인데 xgboost는 데이터를 블록이라는 단위로 정렬하고 압축하여 여러 대의 머신이나 외부 메모리에 분산시킨다. 
  - 캐시 고려 접근: 캐시를 고려한 프리페칭을 사용한다. 
  - 블록 압축, 블록 샤딩
- 정확도를 향상한다.
  - 자체적으로 regularization 추가: 분산을 줄이고 과대적합을 방지한다. 

In [1]:
# pandas와 numpy 임포트하기
import pandas as pd
import numpy as np

# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_bikes = pd.read_csv('../data/bike_rentals_cleaned.csv')
# 데이터를 X와 y로 나눕니다.
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

# train_test_split를 임포트합니다.
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

X_train.head(), y_train.head()

(     instant  season   yr  mnth  holiday  weekday  workingday  weathersit  \
 203      204     3.0  0.0   7.0      0.0      6.0         0.0           1   
 634      635     4.0  1.0   9.0      0.0      3.0         1.0           1   
 544      545     3.0  1.0   6.0      0.0      4.0         1.0           1   
 12        13     1.0  0.0   1.0      0.0      4.0         1.0           1   
 171      172     3.0  0.0   6.0      0.0      2.0         1.0           2   
 
          temp     atemp       hum  windspeed  
 203  0.849167  0.804287  0.500000   0.131221  
 634  0.635000  0.596613  0.630833   0.244400  
 544  0.749167  0.675512  0.422500   0.171650  
 12   0.165000  0.150883  0.470417   0.301000  
 171  0.680833  0.637646  0.770417   0.171025  ,
 203    3285
 634    7733
 544    6879
 12     1406
 171    4835
 Name: cnt, dtype: int64)

In [5]:
# XGBRegressor를 임포트합니다.
from xgboost import XGBRegressor

# mean_squared_error를 임포트합니다.
from sklearn.metrics import mean_squared_error as MSE

# XGBRegressor를 초기화합니다.
xg_reg = XGBRegressor(max_depth=3, n_estimators=1600, eta=0.02, 
                      subsample=0.75, random_state=2)

# 훈련 세트에서 xg_reg를 훈련합니다.
xg_reg.fit(X_train, y_train)

# 테스트 세트에 대한 예측을 만듭니다.
y_pred = xg_reg.predict(X_test)

# 평균 제곱근 오차를 계산합니다.
MSE(y_test, y_pred)**0.5

584.3395337495713